In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from collections import Counter
import ast
import os
import os.path
import csv
from datetime import date, datetime, timedelta
print(os.getcwd())

In [ ]:
stockSymbol = ["AAPL", "AMZN", "GOOGL","MSFT", "DELL", "IBM", "INTC", "HPQ",
               "FB", "CSCO", "ORCL", "HPE", "MU", "DXC", "TMO"]

# initalize relative path directory
script = os.getcwd()
print(script)
images = os.path.join(script, 'visualization', 'WebsitePNGs')
print(images)


# Stock Twits
## Top 5 Other Companies Mentioned in Twits
### Make Pie and Donut Chart

In [ ]:
def MakeDonutChart(data,symbol,timeSeries):
    getCount = Counter(k['symbol'] for k in data if dict(k).get('symbol'))
    symbolCount = dict(getCount)
    symbolCount[symbol] = 0
    symbolCount = {k: v for k, v in sorted(symbolCount.items(), key=lambda item: item[1], reverse=True)}
    fig, ax = plt.subplots(figsize=(11, 10), subplot_kw=dict(aspect="equal"))

    cnt = 0
    data = []
    symbols = []
    recipe = []
    for key in symbolCount.keys():
        data.append(symbolCount[key])
        symbols.append(key)
        recipe.append(key + ' - ' + str(symbolCount[key]) + ' twits')
        cnt += 1
        if cnt >= 5:
            break

    if sum(data) < 15:
        print('Not enough data for given time frame')
        fig.suptitle('Not enough data for given time frame', fontsize=20)
        os.chdir(images)
        plt.savefig(symbol+'TopFiveOtherCompanies'+timeSeries+'.png', optimize=True)
        print('saved '+symbol+'TopFiveOtherCompanies'+timeSeries+'.png to '+os.getcwd())
        plt.close()
        os.chdir(script)
        print('returning to '+os.getcwd())
        return

    def explode():
        try:
            exp = (0.1,0,0,0,0)
        except:
            exp=None
        return(exp)

    wedges, texts = ax.pie(data, 
                           explode=explode(), 
                           shadow=True, wedgeprops=dict(width=0.5), startangle=-40)

    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    kw = dict(arrowprops=dict(arrowstyle="-"),
              bbox=bbox_props, zorder=0, va="center")

    for i, p in enumerate(wedges):
        ang = (p.theta2 - p.theta1)/2. + p.theta1
        y = np.sin(np.deg2rad(ang))
        x = np.cos(np.deg2rad(ang))
        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = "angle,angleA=0,angleB={}".format(ang)
        kw["arrowprops"].update({"connectionstyle": connectionstyle})
        ax.annotate(recipe[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                    horizontalalignment=horizontalalignment, **kw)

    ax.legend(wedges, symbols,
            fontsize='large',
            title_fontsize='large',
            title="Symbols",
            loc="center",
            frameon=False)
            #bbox_to_anchor=(1, 0, 0.5, 1))
    #ax.set_title("Top Ten Companies Mentioned in " + symbol + " Twits", fontsize=30, pad=50)

    os.chdir(images)
    plt.savefig(symbol+'TopFiveOtherCompanies'+timeSeries+'.png', optimize=True)
    print('saved '+symbol+'TopFiveOtherCompanies'+timeSeries+'.png to '+os.getcwd())
    plt.close()
    os.chdir(script)
    print('returning to '+os.getcwd())

### Count Other Companies for Pie and Donut Chart

In [ ]:
def GetOtherCompanies(fname, days='all'):
    to_ignore = fname[:-10]
    # set wd to Symbolfolder
    symbolFolder = os.path.join(script, to_ignore+'folder')
    os.chdir(symbolFolder)
    print('Pulling data from ' +os.getcwd())
    df = pd.read_csv(fname)
    df['Date'] = pd.to_datetime(df['created']) # save time string as datetime
    stock_ds = []
    if days == 'lastWeek':
        lastweekdate = pd.to_datetime('today').floor('D') - timedelta(7)
        df = df[df.Date >= lastweekdate]
        for row in df.newSymbols:
            lists = ast.literal_eval(row)
            for diction in lists:
                stock_ds.append(diction)
        MakeDonutChart(stock_ds, to_ignore, 'LastWeek')
    elif days == 'lastMonth':
        lastmonthdate = pd.to_datetime('today').floor('D') - timedelta(30)
        df = df[df.Date >= lastmonthdate]
        for row in df.newSymbols:
            lists = ast.literal_eval(row)
            for diction in lists:
                stock_ds.append(diction)
        MakeDonutChart(stock_ds, to_ignore, 'LastMonth')
    elif days == 'lastYear':
        lastyeardate = pd.to_datetime('today').floor('D') - timedelta(365)
        df = df[df.Date >= lastyeardate]
        for row in df.newSymbols:
            lists = ast.literal_eval(row)
            for diction in lists:
                stock_ds.append(diction)
        MakeDonutChart(stock_ds, to_ignore, 'LastYear')
    else:
        for row in df.newSymbols:
            lists = ast.literal_eval(row)
            for diction in lists:
                stock_ds.append(diction)
        MakeDonutChart(stock_ds, to_ignore, 'AllTime')

## Bullish/Bearish Tags
### Make Bar Chart

In [ ]:
def MakeBarChart(tags,symbol,timeSeries):
    data = []
    tag = []
    for key in tags.keys():
        data.append(tags[key])
        tag.append(key)
    
    df = pd.DataFrame({'Tags':tag, 'val':data})
    ax = df.plot.barh('Tags', 'val', color=['g', 'r', 'y'], fontsize=15, figsize=(11,10), legend=False)
    ax.set_ylabel('Tags', fontsize=20)
    for i, v in enumerate(data):
        ax.text(v, i, str(v), fontsize=15, fontweight='bold')
    
    os.chdir(images)
    plt.savefig(symbol+'Tags'+timeSeries+'.png', optimize=True)
    print('saved '+symbol+'Tags'+timeSeries+'.png to '+os.getcwd())
    plt.close()
    os.chdir(script)
    print('returning to '+os.getcwd())

### Count Tags for Bar Chart

In [ ]:
def GetTags(fname, days='all'):
    symbol = fname[:-10]
    # set wd to Symbolfolder
    symbolFolder = os.path.join(script, symbol+'folder')
    os.chdir(symbolFolder)
    print('Pulling data from ' +os.getcwd())
    df = pd.read_csv(fname)
    df['Date'] = pd.to_datetime(df['created']) # save time string as datetime
    tags = {
        'Bullish': 0,
        'Bearish': 0,
        'none': 0
    }

    if days == 'lastWeek':
        lastweekdate = pd.to_datetime('today').floor('D') - timedelta(7)
        df = df[df.Date >= lastweekdate]
        for row in df.tag:
            tags[row] += 1
        MakeBarChart(tags,symbol,'LastWeek')
    elif days == 'lastMonth':
        lastmonthdate = pd.to_datetime('today').floor('D') - timedelta(30)
        df = df[df.Date >= lastmonthdate]
        for row in df.tag:
            tags[row] += 1
        MakeBarChart(tags,symbol,'LastMonth')
    elif days == 'lastYear':
        lastyeardate = pd.to_datetime('today').floor('D') - timedelta(365)
        df = df[df.Date >= lastyeardate]
        for row in df.tag:
            tags[row] += 1
        MakeBarChart(tags,symbol,'LastYear')
    else:
        for row in df.tag:
            tags[row] += 1
        MakeBarChart(tags,symbol,'AllTime')

# Daily and Minute Values
## Stock Performance

In [ ]:
def GetVolume(fname, days='all'):
    if 'Values' in fname:
        symbol = fname[:-10]
    else:
        symbol = fname[:-9]
    symbolFolder = os.path.join(script, symbol+'folder')
    os.chdir(symbolFolder)
    print('Pulling data from ' +os.getcwd())
    df = pd.read_csv(fname)
    df['Date'] = pd.to_datetime(df['time'])
    df["SMA1"] = df['close'].rolling(window=25).mean()
    df["SMA2"] = df['close'].rolling(window=100).mean()
    
    if days == 'lastWeek':
        lastweekdate = pd.to_datetime('today').floor('D') - timedelta(7)
        df = df[df.Date >= lastweekdate]
        ax = df.plot('Date', 'volume', figsize=(15,10), fontsize=15)
        ax.set_xlabel('')
        ax.set_ylabel('Volume', fontsize=20)
        plt.ticklabel_format(style='plain', axis='y')
        plt.grid(True)
        os.chdir(images)
        plt.savefig(symbol+'VolumeLastWeek.png', optimize=True)
        print('saved '+symbol+'VolumeLastWeek.png to '+os.getcwd())
        plt.close()
        os.chdir(symbolFolder)
        print('returning to '+os.getcwd())
        ax = df.plot('Date', 'close', figsize=(11,10), fontsize=15)
        ax.set_xlabel('')
        ax.set_ylabel('Price', fontsize=20)
        plt.legend()
        plt.ticklabel_format(style='plain', axis='y')
        plt.grid(True)
        os.chdir(images)
        plt.savefig(symbol+'PriceLastWeek.png', optimize=True)
        print('saved '+symbol+'PriceLastWeek.png to '+os.getcwd())
        plt.close()
        os.chdir(script)
        print('Done - returning to '+os.getcwd())

    elif days == 'lastMonth':
        lastmonthdate = pd.to_datetime('today').floor('D') - timedelta(30)
        df = df[df.Date >= lastmonthdate]
        ax = df.plot('Date', 'volume', figsize=(15,10), fontsize=15)
        ax.set_xlabel('')
        ax.set_ylabel('Volume', fontsize=20)
        plt.ticklabel_format(style='plain', axis='y')
        plt.grid(True)
        os.chdir(images)
        plt.savefig(symbol+'VolumeLastMonth.png', optimize=True)
        print('saved '+symbol+'VolumeLastMonth.png to '+os.getcwd())
        os.chdir(symbolFolder)
        print('returning to '+os.getcwd())
        plt.close()
        ax = df.plot('Date', 'close', figsize=(11,10), fontsize=15)
        ax.set_xlabel('')
        ax.set_ylabel('Price', fontsize=20)
        plt.ticklabel_format(style='plain', axis='y')
        plt.grid(True)
        os.chdir(images)
        plt.savefig(symbol+'PriceLastMonth.png', optimize=True)
        print('saved '+symbol+'PriceLastMonth.png to '+os.getcwd())
        os.chdir(script)
        print('Done - returning to '+os.getcwd())
        plt.close()

    elif days == 'lastYear':
        lastyeardate = pd.to_datetime('today').floor('D') - timedelta(365)
        df = df[df.Date >= lastyeardate]
        ax = df.plot('Date', 'volume', figsize=(15,10), fontsize=15)
        ax.set_xlabel('')
        ax.set_ylabel('Volume', fontsize=20)
        plt.ticklabel_format(style='plain', axis='y')
        plt.grid(True)
        os.chdir(images)
        plt.savefig(symbol+'VolumeLastYear.png', optimize=True)
        print('saved '+symbol+'VolumeLastYear.png to '+os.getcwd())
        os.chdir(symbolFolder)
        print('returning to '+os.getcwd())
        plt.close()
        ax = df.plot('Date', 'close', figsize=(11,10), fontsize=15)
        plt.plot(df.Date, df['SMA1'], 'g--', label="Simple Moving Average - 25 Days")
        plt.plot(df.Date, df['SMA2'], 'r--', label="Simple Moving Average - 100 Days")
        ax.set_xlabel('')
        ax.set_ylabel('Price', fontsize=20)
        plt.legend()
        plt.ticklabel_format(style='plain', axis='y')
        plt.grid(True)
        os.chdir(images)
        plt.savefig(symbol+'PriceLastYear.png', optimize=True)
        print('saved '+symbol+'PriceLastYear.png to '+os.getcwd())
        plt.close()
        os.chdir(script)
        print('Done - returning to '+os.getcwd())
        
    else:
        ax = df.plot('Date', 'volume', figsize=(15,10), fontsize=15)
        ax.set_xlabel('')
        ax.set_ylabel('Volume', fontsize=20)
        plt.ticklabel_format(style='plain', axis='y')
        plt.grid(True)
        os.chdir(images)
        plt.savefig(symbol+'VolumeAllTime.png', optimize=True)
        print('saved '+symbol+'VolumeAllTime.png to '+os.getcwd())
        plt.close()
        os.chdir(symbolFolder)
        print('returning to '+os.getcwd())
        ax = df.plot('Date', 'close', figsize=(11,10), fontsize=15)
        plt.plot(df.Date, df['SMA1'], 'g--', label="Simple Moving Average - 25 Days")
        plt.plot(df.Date, df['SMA2'], 'r--', label="Simple Moving Average - 100 Days")
        ax.set_xlabel('')
        ax.set_ylabel('Price', fontsize=20)
        plt.legend()
        plt.ticklabel_format(style='plain', axis='y')
        plt.grid(True)
        os.chdir(images)
        plt.savefig(symbol+'PriceAllTime.png', optimize=True)
        print('saved '+symbol+'PriceAllTime.png to '+os.getcwd())
        os.chdir(script)
        print('Done - returning to '+os.getcwd())
        plt.close()

# Create Visualizations

In [ ]:
for i in stockSymbol:
    GetOtherCompanies('{}_twits.csv'.format(i), days='lastWeek')
    GetOtherCompanies('{}_twits.csv'.format(i), days='lastMonth')
    GetOtherCompanies('{}_twits.csv'.format(i), days='lastYear')
    GetOtherCompanies('{}_twits.csv'.format(i))
    GetVolume('{}Daily.csv'.format(i), days='lastWeek')
    GetVolume('{}Daily.csv'.format(i), days='lastMonth')
    GetVolume('{}Daily.csv'.format(i), days='lastYear')
    GetVolume('{}Daily.csv'.format(i))
    GetTags('{}_twits.csv'.format(i), days='lastWeek')
    GetTags('{}_twits.csv'.format(i), days='lastMonth')
    GetTags('{}_twits.csv'.format(i), days='lastYear')
    GetTags('{}_twits.csv'.format(i))

## AAPL

In [ ]:
# GetOtherCompanies('AAPL_Twits.csv', days='lastWeek')
# GetOtherCompanies('AAPL_Twits.csv', days='lastMonth')
# GetOtherCompanies('AAPL_Twits.csv', days='lastYear')
# GetOtherCompanies('AAPL_Twits.csv')
GetVolume('AAPLDaily.csv', days='lastWeek')
GetVolume('AAPLDaily.csv', days='lastMonth')
GetVolume('AAPLDaily.csv', days='lastYear')
GetVolume('AAPLDaily.csv')
GetTags('AAPL_Twits.csv', days='lastWeek')
GetTags('AAPL_Twits.csv', days='lastMonth')
GetTags('AAPL_Twits.csv', days='lastYear')
GetTags('AAPL_Twits.csv')

## AMZN

In [ ]:
GetOtherCompanies('AMZN_Twits.csv', days='lastWeek')
GetOtherCompanies('AMZN_Twits.csv', days='lastMonth')
GetOtherCompanies('AMZN_Twits.csv', days='lastYear')
GetOtherCompanies('AMZN_Twits.csv')
GetVolume('AMZNDaily.csv', days='lastWeek')
GetVolume('AMZNDaily.csv', days='lastMonth')
GetVolume('AMZNDaily.csv', days='lastYear')
GetVolume('AMZNDaily.csv')

## GOOGL

In [ ]:
GetOtherCompanies('GOOGL_Twits.csv', days='lastWeek')
GetOtherCompanies('GOOGL_Twits.csv', days='lastMonth')
GetOtherCompanies('GOOGL_Twits.csv', days='lastYear')
GetOtherCompanies('GOOGL_Twits.csv')
GetVolume('GOOGLDaily.csv', days='lastWeek')
GetVolume('GOOGLDaily.csv', days='lastMonth')
GetVolume('GOOGLDaily.csv', days='lastYear')
GetVolume('GOOGLDaily.csv')

## MSFT

In [ ]:
GetOtherCompanies('MSFT_Twits.csv', days='lastWeek')
GetOtherCompanies('MSFT_Twits.csv', days='lastMonth')
GetOtherCompanies('MSFT_Twits.csv', days='lastYear')
GetOtherCompanies('MSFT_Twits.csv')
GetVolume('MSFTDaily.csv', days='lastWeek')
GetVolume('MSFTDaily.csv', days='lastMonth')
GetVolume('MSFTDaily.csv', days='lastYear')
GetVolume('MSFTDaily.csv')

## DELL

In [ ]:
GetOtherCompanies('DELL_Twits.csv', days='lastWeek')
GetOtherCompanies('DELL_Twits.csv', days='lastMonth')
GetOtherCompanies('DELL_Twits.csv', days='lastYear')
GetOtherCompanies('DELL_Twits.csv')
GetVolume('DELLDaily.csv', days='lastWeek')
GetVolume('DELLDaily.csv', days='lastMonth')
GetVolume('DELLDaily.csv', days='lastYear')
GetVolume('DELLDaily.csv')

## IBM

In [ ]:
GetOtherCompanies('IBM_Twits.csv', days='lastWeek')
GetOtherCompanies('IBM_Twits.csv', days='lastMonth')
GetOtherCompanies('IBM_Twits.csv', days='lastYear')
GetOtherCompanies('IBM_Twits.csv')
GetVolume('IBMDaily.csv', days='lastWeek')
GetVolume('IBMDaily.csv', days='lastMonth')
GetVolume('IBMDaily.csv', days='lastYear')
GetVolume('IBMDaily.csv')

## INTC

In [ ]:
GetOtherCompanies('INTC_Twits.csv', days='lastWeek')
GetOtherCompanies('INTC_Twits.csv', days='lastMonth')
GetOtherCompanies('INTC_Twits.csv', days='lastYear')
GetOtherCompanies('INTC_Twits.csv')
GetVolume('INTCDaily.csv', days='lastWeek')
GetVolume('INTCDaily.csv', days='lastMonth')
GetVolume('INTCDaily.csv', days='lastYear')
GetVolume('INTCDaily.csv')

## HPQ

In [ ]:
GetOtherCompanies('HPQ_Twits.csv', days='lastWeek')
GetOtherCompanies('HPQ_Twits.csv', days='lastMonth')
GetOtherCompanies('HPQ_Twits.csv', days='lastYear')
GetOtherCompanies('HPQ_Twits.csv')
GetVolume('HPQDaily.csv', days='lastWeek')
GetVolume('HPQDaily.csv', days='lastMonth')
GetVolume('HPQDaily.csv', days='lastYear')
GetVolume('HPQDaily.csv')

## FB

In [ ]:
GetOtherCompanies('FB_Twits.csv', days='lastWeek')
GetOtherCompanies('FB_Twits.csv', days='lastMonth')
GetOtherCompanies('FB_Twits.csv', days='lastYear')
GetOtherCompanies('FB_Twits.csv')
GetVolume('FBDaily.csv', days='lastWeek')
GetVolume('FBDaily.csv', days='lastMonth')
GetVolume('FBDaily.csv', days='lastYear')
GetVolume('FBDaily.csv')

## CSCO

In [ ]:
GetOtherCompanies('CSCO_Twits.csv', days='lastWeek')
GetOtherCompanies('CSCO_Twits.csv', days='lastMonth')
GetOtherCompanies('CSCO_Twits.csv', days='lastYear')
GetOtherCompanies('CSCO_Twits.csv')
GetVolume('CSCODaily.csv', days='lastWeek')
GetVolume('CSCODaily.csv', days='lastMonth')
GetVolume('CSCODaily.csv', days='lastYear')
GetVolume('CSCODaily.csv')

## ORCL

In [ ]:
GetOtherCompanies('ORCL_Twits.csv', days='lastWeek')
GetOtherCompanies('ORCL_Twits.csv', days='lastMonth')
GetOtherCompanies('ORCL_Twits.csv', days='lastYear')
GetOtherCompanies('ORCL_Twits.csv')
GetVolume('ORCLDaily.csv', days='lastWeek')
GetVolume('ORCLDaily.csv', days='lastMonth')
GetVolume('ORCLDaily.csv', days='lastYear')
GetVolume('ORCLDaily.csv')

## HPE

In [ ]:
GetOtherCompanies('HPE_Twits.csv', days='lastWeek')
GetOtherCompanies('HPE_Twits.csv', days='lastMonth')
GetOtherCompanies('HPE_Twits.csv', days='lastYear')
GetOtherCompanies('HPE_Twits.csv')
GetVolume('HPEDaily.csv', days='lastWeek')
GetVolume('HPEDaily.csv', days='lastMonth')
GetVolume('HPEDaily.csv', days='lastYear')
GetVolume('HPEDaily.csv')

## MU

In [ ]:
GetOtherCompanies('MU_Twits.csv', days='lastWeek')
GetOtherCompanies('MU_Twits.csv', days='lastMonth')
GetOtherCompanies('MU_Twits.csv', days='lastYear')
GetOtherCompanies('MU_Twits.csv')
GetVolume('MUDaily.csv', days='lastWeek')
GetVolume('MUDaily.csv', days='lastMonth')
GetVolume('MUDaily.csv', days='lastYear')
GetVolume('MUDaily.csv')

## DXC

In [ ]:
GetOtherCompanies('DXC_Twits.csv', days='lastWeek')
GetOtherCompanies('DXC_Twits.csv', days='lastMonth')
GetOtherCompanies('DXC_Twits.csv', days='lastYear')
GetOtherCompanies('DXC_Twits.csv')
GetVolume('DXCDaily.csv', days='lastWeek')
GetVolume('DXCDaily.csv', days='lastMonth')
GetVolume('DXCDaily.csv', days='lastYear')
GetVolume('DXCDaily.csv')

## TMO

In [ ]:
GetOtherCompanies('TMO_Twits.csv', days='lastWeek')
GetOtherCompanies('TMO_Twits.csv', days='lastMonth')
GetOtherCompanies('TMO_Twits.csv', days='lastYear')
GetOtherCompanies('TMO_Twits.csv')
GetVolume('TMODaily.csv', days='lastWeek')
GetVolume('TMODaily.csv', days='lastMonth')
GetVolume('TMODaily.csv', days='lastYear')
GetVolume('TMODaily.csv')
GetTags('TMO_Twits.csv', days='lastWeek')
GetTags('TMO_Twits.csv', days='lastMonth')
GetTags('TMO_Twits.csv', days='lastYear')
GetTags('TMO_Twits.csv')